In [71]:
from sklearn.tree import DecisionTreeRegressor
from pandas import read_csv
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import pickle
import os 
import os.path
from pathlib import Path
from sklearn import datasets
from sklearn import tree, model_selection
from scipy.stats import spearmanr
import math
import numpy as np



path = Path.cwd()
path = path.parent.absolute()
path = Path.joinpath(path, 'data', 'DB covidiagnostix_aggiornatopulito.xlsx')

df = pd.read_excel(path)

df1 = pd.DataFrame(df, columns= ['TITOLO Ab/ TEMPO 0 (prima del vaccino)'])
df2 = pd.DataFrame(df, columns= ['TITOLO Ab/ TEMPO 21 (2 dose) (INDEX) SIEMENS U/ML'])
df3 = pd.DataFrame(df, columns= ['TITOLO Ab dopo 2 MESI dalla 2 dose (INDEX) U/ML SIEMENS'])
df4 = pd.DataFrame(df, columns= ['TITOLO Ab dopo 5/6 MESI dalla 2 dose (INDEX) SIEMENS U/ML'])

spear, _ = spearmanr(df1, df2)
print (f'coefficiente di correlazione di Spearman tra i titoli a tempo 0 e tempo 21 (Siemens): \n"{spear:.3f}')

spear, _ = spearmanr(df1, df3)
print (f'coefficiente di correlazione di Spearman tra i titoli a tempo 0 e passati 2 mesi dalla 2 dose (Siemens): \n"{spear:.3f}')

spear, _ = spearmanr(df1, df4)
print (f'coefficiente di correlazione di Spearman tra i titoli a tempo 0 e passati 5/6 mesi dalla 2 dose (Siemens): \n"{spear:.3f}')

spear, _ = spearmanr(df2, df3)
print (f'coefficiente di correlazione di Spearman tra i titoli a tempo 21 e passati 2 mesi dalla 2 dose (Siemens): \n"{spear:.3f}')

spear, _ = spearmanr(df2, df4)
print (f'coefficiente di correlazione di Spearman tra i titoli a tempo 21 e passati 5/6 mesi dalla 2 dose (Siemens): \n"{spear:.3f}')

spear, _ = spearmanr(df3, df4)
print (f'coefficiente di correlazione di Spearman tra i titoli passati 2 mesi dalla 2 dose e passati 5/6 mesi dalla 2 dose (Siemens): \n"{spear:.3f}')






array = df.values 
siemens = df.iloc[: , [0,2,4]]
targetSiemens = df.iloc [:, 6]

targetSiemensStd = targetSiemens.std()


print (f'std siemens : {targetSiemensStd:.3f}')
X_train, X_test, y_train, y_test = train_test_split(siemens, targetSiemens, test_size=0.20, random_state=3)

dt = DecisionTreeRegressor(max_depth = 4, min_samples_leaf = 0.1, random_state = 3)

kfold = model_selection.KFold(n_splits=10)


dt.fit(X_train, y_train)

y_pred_test = dt.predict (X_test)

mse_dt = MSE(y_test, y_pred_test)

rmse_dt = mse_dt**(1/2)


scoring = "neg_mean_absolute_error"
results = model_selection.cross_val_score(dt, X_train, y_train, cv=kfold, scoring=scoring)
        
#mae1 = MAE(y_test, y_pred_test) 

        
#print(f'Mean Absolute Error: {mae1:.3f}')


resultsStd = results.std()

print(f'Mean Absolute Error: {abs(results.mean()):.3f}'); 
print(f'Standard Deviation:  {resultsStd:.3f}')
print (f'this is the rmse : {rmse_dt:.3f}')
     
rapportoMaeStd = results.mean()/targetSiemensStd
print (f'mae/stdY : {abs(rapportoMaeStd):.3f}')
    
      
      
# save the model to disk
filename = 'finalized_model_siemens.sav'
pickle.dump(dt, open(filename, 'wb'))

# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(f'Result of the model : {result:.3f}')

print('Media di ogni dato :')

x = round(df.mean(), 3)

print (x)
       
print('Mediana di ogni dato :')
print (df.median())

coefficiente di correlazione di Spearman tra i titoli a tempo 0 e tempo 21 (Siemens): 
"0.680
coefficiente di correlazione di Spearman tra i titoli a tempo 0 e passati 2 mesi dalla 2 dose (Siemens): 
"0.548
coefficiente di correlazione di Spearman tra i titoli a tempo 0 e passati 5/6 mesi dalla 2 dose (Siemens): 
"0.580
coefficiente di correlazione di Spearman tra i titoli a tempo 21 e passati 2 mesi dalla 2 dose (Siemens): 
"0.788
coefficiente di correlazione di Spearman tra i titoli a tempo 21 e passati 5/6 mesi dalla 2 dose (Siemens): 
"0.715
coefficiente di correlazione di Spearman tra i titoli passati 2 mesi dalla 2 dose e passati 5/6 mesi dalla 2 dose (Siemens): 
"0.844
std siemens : 98.388
Mean Absolute Error: 27.040
Standard Deviation:  7.301
this is the rmse : 98.768
mae/stdY : 0.275
Result of the model : 0.155
Media di ogni dato :
TITOLO Ab/ TEMPO 0 (prima del vaccino)                          1.492
TITOLO Ab/ TEMPO 0 (prima del vaccino) DIESSE                 104.633
TITOLO 